# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib notebook


In [2]:
from itertools import product
import os

import graph_tool.all as gt
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import hypergeom, pearsonr
import seaborn as sns
from sklearn.cluster import KMeans

from functions import *


# Graph-Tool compatibility
plt.switch_backend('cairo')
# Style
sns.set_theme(context='talk', style='white', palette='Set2')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


# Metadata

In [3]:
# Load metadata
meta = get_meta()

# Subject preview
filtered = []
for i, row in meta.iterrows():
    try:
        load_graph_by_id(row['SubID'])
        assert not np.isnan(row['nps_MoodDysCurValue'])  # Has NPS information available
        assert row['BRAAK_AD'] in (6,) and row['CERAD'] in (4,) and row['CDRScore'] in (3,)
    except:
        continue
    filtered.append(f'{row["SubID"]} {row["Ethnicity"]} {row["Sex"]}, {row["Age"]}, BRAAK {row["BRAAK_AD"]}, CERAD {row["CERAD"]}, CDR {row["CDRScore"]}, {row["Dx"]}')
filtered = np.sort(filtered)
for i in range(len(filtered)):
    # print(filtered[i])
    pass


/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/file.py:122: DtypeWarning: Columns (5,157,158,159,161,164,165,166,167,170,172,176,177,178,179,189,191,193,195,276,285,286,287,288,289,290,291,292,293,294,295,297,342,345,346,462,465,566,570,571,572,573,574,580,582,583,584,586,589,592,594,597,599,601,603,606,607,611,613,615,617,618,620,622,625,684,686) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(META)


# Attention Stack

In [4]:
fname = './attentions.pkl'
if os.path.isfile(fname):
    # Load data
    with open('./attentions.pkl', 'rb') as f:
        all_data = pickle.load(f)
    attention_stack, all_edges, columns, subject_ids = all_data['data'], all_data['edges'], all_data['heads'], all_data['subject_ids']

else:
    # Parameters
    # Scaled probably shouldn't be used, but better for visualization
    # until results are more even
    columns = get_attention_columns(scaled=False)
    subject_ids = meta['SubID'].to_numpy()

    # Load graphs
    graphs, subject_ids = load_many_graphs(subject_ids, column=columns)
    # graphs = [compute_graph(g) for g in graphs]

    # # Get attentions
    # df = {}
    # for column in get_attention_columns():
    #     attention, _ = compute_edge_summary(graphs, subject_ids=subject_ids)
    #     attention = attention.set_index('Edge')
    #     df[column] = attention.var(axis=1)


    # Set indices to edges and clean
    print('Fixing indices...')
    for i in tqdm(range(len(graphs))):
        graphs[i].index = graphs[i].apply(lambda r: get_edge_string([r['TF'], r['TG']]), axis=1)
        graphs[i] = graphs[i].drop(columns=['TF', 'TG'])
        # Remove duplicates
        graphs[i] = graphs[i][~graphs[i].index.duplicated(keep='first')]

    # Get all unique edges
    print('Getting unique edges...')
    all_edges = np.unique(sum([list(g.index) for g in graphs], []))


    # Standardize index order
    print('Standardizing indices...')
    for i in tqdm(range(len(graphs))):
        # Add missing indices and order based on `all_edges`
        # to_add = [edge for edge in all_edges if edge not in list(graphs[i].index)]  # SLOW
        to_add = list(set(all_edges) - set(graphs[i].index))

        # Empty rows
        new_rows = pd.DataFrame(
            [[np.nan]*len(graphs[i].columns)]*len(to_add),
            columns=graphs[i].columns,
        ).set_index(pd.Series(to_add))
        # Native concat
        graphs[i] = pd.concat([graphs[i], new_rows]).loc[all_edges]

    # Convert to numpy
    graphs = [g.to_numpy() for g in graphs]
    attention_stack = np.stack(graphs, axis=-1)
    # attention_stack.shape = (Edge, Head, Subject)
    # attention_stack.shape = (all_edges, columns, subject_ids)

    # Save all data
    all_data = {'data': attention_stack, 'edges': all_edges, 'heads': columns, 'subject_ids': subject_ids}
    # np.savez('attentions.npz', **all_data)
    with open(fname, 'wb') as f:
        pickle.dump(
            all_data,
            f,
            protocol=pickle.HIGHEST_PROTOCOL,
        )


In [5]:
# Additional useful parameters
self_loops = [split_edge_string(s)[0] == split_edge_string(s)[1] for s in all_edges]
self_loops = np.array(self_loops)
# Remove self loops
all_edges = all_edges[~self_loops]
attention_stack = attention_stack[~self_loops]


# Global Parameters

In [6]:
# Parameters
print(f'\nAvailable attention columns: {get_attention_columns()}')
column_ad = get_attention_columns()[0]
column_scz = get_attention_columns()[2]
column_data = get_attention_columns()[4]
synthetic_nodes_of_interest = ['OPC', 'Micro', 'Oligo']



Available attention columns: ['att_D_AD_1', 'att_D_AD_2', 'att_D_SCZ_1', 'att_D_SCZ_2', 'att_D_no_prior_0', 'att_D_no_prior_1', 'att_D_no_prior_2', 'att_D_no_prior_3']


# Intra-Contrast Comparisons (Figure 3)

In [7]:
# Figure parameters
param = {
    'subjects': ['M35115', 'M35594'],
    'columns': [column_data, column_ad, column_scz],
    'column_names': ['Data Prioritization', 'AD Prioritization', 'SCZ Prioritization'],
    'contrast': 'c15x',
}

# Generate palette
palette = plt.rcParams['axes.prop_cycle'].by_key()['color']
param['palette'] = {sid: rgba_to_hex(palette[i]) for i, sid in enumerate(param['subjects'])}

# Create figure
shape = """
    ..AAAAAABBBBBBBBBBBB
    ..AAAAAABBBBBBBBBBBB
    ..AAAAAABBBBBBBBBBBB
    ..AAAAAABBBBBBBBBBBB
    ..AAAAAABBBBBBBBBBBB
    ..AAAAAACCCCCCCCCCCC
    ..AAAAAACCCCCCCCCCCC
    ..AAAAAACCCCCCCCCCCC
    ..DDDDDDEEEEFFFFGGGG
    ..DDDDDDEEEEFFFFGGGG
    ..DDDDDDEEEEFFFFGGGG
    ..DDDDDDHHHHHHHHHHHH
    ..DDDDDDHHHHHHHHHHHH
    ..DDDDDDHHHHHHHHHHHH
    ..DDDDDDHHHHHHHHHHHH
    ..DDDDDDHHHHHHHHHHHH
    IIIIIJJJJJKKKKKKKKKK
    IIIIIJJJJJKKKKKKKKKK
    IIIIIJJJJJKKKKKKKKKK
    IIIIIJJJJJKKKKKKKKKK
    IIIIIJJJJJKKKKKKKKKK
    IIIIIJJJJJKKKKKKKKKK
    IIIIIJJJJJKKKKKKKKKK
"""
shape_array = [s.replace(' ', '') for s in shape.split('\n')[1:-1]]
shape_array = np.array([list(s) for s in shape_array])


In [24]:
# Subplot layout (doesn't work well with constrained layout)
# NOTE: This cannot be used, as constrained layout has glitches
# (see https://github.com/matplotlib/matplotlib/issues/23290)
# with uneven mosaics
# fig, axs = get_mosaic(shape, figsize=(int((3/2) * shape_array.shape[1]), int((3/2) * shape_array.shape[0])), constrained_layout=False)

# Subfigure layout (longer)
# NOTE: Constrained layout will fail for all
# subplots if a single one is not able to scale
fig, axs = create_subfigure_mosaic(shape_array)


# Plot all panels

print('Individual Panels (A, D)')
axs['A'].clear(); axs['D'].clear()
plot_graph_comparison_from_sids(param['subjects'], axs={0: axs['A'], 1: axs['D']}, column=param['columns'][0], vertex_ids=synthetic_nodes_of_interest)
# Legend
plot_legend(horizontal=False, loc='center', bbox_to_anchor=(.5, -.1), ax=axs['A'])


print('\nCell Type Priority (B)')
plot_ct_individual_edges(param['subjects'][0], **all_data, column=param['columns'][0], ax=axs['B'])
axs['B'].set_title(None)


print('\nAttention Histogram (C)')
plot_attention_histogram(param['subjects'][0], **all_data, column=param['columns'][0], ax=axs['C'])
axs['C'].set_xlabel(param['column_names'][0])


print('\nEdge Comparisons (E, F, G)')
for ax, column, title in zip([axs['E'], axs['F'], axs['G']], param['columns'], param['column_names']):
    ax.clear()
    plot_edge_comparison_from_sids(param['subjects'], ax=ax, column=column, palette=param['palette'], highlight_outliers=True)
    ax.set_title(title)


print('\nModule Analysis (H)')
plot_module_scores_from_sids(param['subjects'], ax=axs['H'], palette=param['palette'], column=param['columns'][0])
axs['H'].set_title(param['column_names'][0])


print('\nCell Type Linkage Analysis (I)')
plot_ct_edges(param['contrast'], **all_data, column=param['columns'][0], ax=axs['I'])
axs['I'].set_title(param['column_names'][0])


print('\nEdge Discovery (J)')
plot_edge_discovery(**all_data, column=param['columns'][0], ax=axs['J'])
axs['J'].set_title(param['column_names'][0])


print('\nEnrichment (K)')
# Compute gene list
fname = f'../plots/genes_{"_".join(param["subjects"])}.csv'
if not os.path.isfile(fname):
    df = compute_all_important_genes_from_sids(param['subjects'], **all_data, vertex_ids=synthetic_nodes_of_interest)
    df.to_csv(fname, index=False)

# MANUAL PROCESSING
# Run the output from above on Metascape as multiple gene list and perform
# enrichment.  From the all-in-one ZIP file, save the file from
# Enrichment_QC/HeatmapSelectedGOTop100.csv as `fname` below

# Read enrichment
fname = f'../plots/go_{"_".join(param["subjects"])}.csv'
if os.path.isfile(fname):
    plot_enrichment_from_fname(fname, ax=axs['K'])
axs['K'].set_xlabel(None)


# Save figure
print()
fig.savefig(f'../plots/figure_3.pdf', format='pdf', bbox_inches='tight', transparent=True, backend='cairo')


Individual Panels (A, D)
Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████████████| 378/378 [00:00<00:00, 336827.47it/s]


Calculating positions...
Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████████████| 171/171 [00:00<00:00, 288390.02it/s]


Removing duplicate edges...


...


Module Analysis (H)


/home/thema/miniconda3/envs/GNN/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)



Cell Type Linkage Analysis (I)


/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/plotting.py:1150: RuntimeWarning: Mean of empty slice
  group_means[group] = np.nanmean(filtered_data, axis=2).flatten()



Edge Discovery (J)

Enrichment (K)



In [23]:
# Make label overlay
# NOTE: Cannot be done on main plot, as constrained layout does not support it
fig, axs = create_subfigure_mosaic(shape_array, layout=None)
for label, ax in axs.items():
    # Clear axis
    ax.axis('off')
    # Figure level
    pos = ax.figure.transSubfigure.transform((0, 1)) + 20 * np.array([1, -1])
    pos = fig.transFigure.inverted().transform(pos)
    plt.text(*pos, label, ha='right', va='bottom', fontweight='bold', fontsize='xx-large', transform=fig.transFigure)
    # Subfigure level
    # pos = ax.figure.transSubfigure.inverted().transform(ax.figure.transSubfigure.transform((0, 1)) + 20 * np.array([1, -1]))
    # plt.text(*pos, label, ha='right', va='bottom', fontweight='bold', fontsize='xx-large', transform=ax.figure.transSubfigure)
fig.savefig(f'../plots/figure_3_labels.pdf', format='pdf', bbox_inches='tight', transparent=True)


# Inter-Contrast Comparisons (Figure 4)

In [20]:
# # Combinations
# # TODO: Potentially move each entry to dictionary, so changes in order
# #   are easier to propagate
# contrast_groupings = [
#     # (contrast name, contrast group, attention column, comparison column, target meta column, other target meta column)
#     # for contrast_name, contrast_group, column, comparison, target, target_comparison in contrast_groupings:
#     # TODO: Revise ethnicity prediction
#     ('c15x', 'AD', column_ad, column_data, 'BRAAK_AD', 'Ethnicity'),
#     ('c15x', 'AD', column_data, column_ad, 'BRAAK_AD', 'Ethnicity'),
#     ('c15x', 'AD', column_data, column_ad, 'nps_MoodDysCurValue', 'nps_WtGainCurValue'),
#     # ('c06x', 'AD', column_ad, column_data, 'BRAAK_AD', 'nps_MoodDysCurValue'),  # Eventually SCZ, BP and such
#     # ('c71x', 'MoodDys', column_data, column_ad, 'nps_MoodDysCurValue'),  # Dysphoria
#     # ('c72x', 'DecInt', column_data, column_ad, 'nps_DecIntCurValue'),  # Anhedonia
# ]


### 4B Distribution Comparison

In [21]:
# for contrast_name, _, column, comparison, target, target_comparison in contrast_groupings:
#     # Filter attention stack to contrast
#     contrast = get_contrast(contrast_name)
#     contrast_subject_ids = sum([contrast[group] for group in contrast], [])
#     contrast_mask = [sid in contrast_subject_ids for sid in subject_ids]
#     contrast_subject_ids = np.array(subject_ids)[contrast_mask]
#     contrast_stack = attention_stack[:, :, contrast_mask]

#     # Filter to 1000 most variant edges
#     top_variant_edge_idx = np.nan_to_num(
#         contrast_stack[:, np.argwhere(np.array(columns)==column)[0][0]]).var(axis=1).argsort()[::-1][:1000]
#     contrast_stack = contrast_stack[top_variant_edge_idx]
#     edge_names = all_edges[top_variant_edge_idx]

#     # Correlation df
#     df = pd.DataFrame(
#         contrast_stack[:, np.argwhere(np.array(columns)==column)[0][0]],
#         index=pd.Series(all_edges[top_variant_edge_idx]),
#         columns=contrast_subject_ids).T
#     df = df.join(meta.set_index('SubID')[[target, target_comparison]]).reset_index(drop=True)
#     # Select edge which most cleanly separates `target`
#     # top_distinct_edge_idx = df.drop(target_comparison, axis=1).groupby(target).mean().var(axis=0).argsort()[-1]
#     # Select top 3 most correlating edges
#     edge_name = df.drop(target_comparison, axis=1).corr()[target].abs().drop(target).sort_values(ascending=False)[:3].index.to_numpy()
#     top_distinct_edge_idx = [np.argwhere(df.columns==edge)[0][0] for edge in edge_name]
#     contrast_stack = contrast_stack[top_distinct_edge_idx]

#     # Plot
#     fig, axs = get_mosaic(np.array(sum([[i]*2 for i in range(6)], [])).reshape((2, -1)), scale=5)
#     # axs[0].sharex(axs[1])
#     sns.despine()

#     for i in range(3):
#         # Filter
#         contrast_stack_i = contrast_stack[i]
#         edge_name_i = edge_name[i]

#         # Scale attention
#         # TODO: Remove once heads are balanced
#         contrast_stack_i = contrast_stack_i / np.nan_to_num(contrast_stack_i).max(axis=1).reshape((-1, 1))

#         # Format
#         df = pd.DataFrame(contrast_stack_i, index=pd.Series(columns), columns=contrast_subject_ids)
#         df = df.reset_index(names='Head').melt(id_vars='Head', var_name='Subject', value_name=edge_name_i).dropna()  # Melt
#         df = df.set_index('Subject').join(meta.set_index('SubID')[[target, target_comparison]]).reset_index()  # Join meta

#         # Filter to target heads
#         df = df.loc[df['Head'].apply(lambda s: s in (column, comparison))]

#         # Main target
#         p1 = sns.violinplot(data=df, x='Head', y=edge_name_i, hue=target, ax=axs[i])
#         p1.legend(title=target, bbox_to_anchor=(1.1, 1.05))
#         p1.set(xlabel=None, xticklabels=[])

#         # Comparison target
#         p2 = sns.violinplot(data=df, x='Head', y=edge_name_i, hue=target_comparison, ax=axs[i+3])
#         p2.legend(title=target_comparison, bbox_to_anchor=(1.1, 1.05))
#         plt.sca(p2)
#         plt.xticks(rotation=60)

#         # Get correlation p-values for targets (which must be numeric)
#         for j, tar in enumerate((target, target_comparison)):
#             for k, c in enumerate(np.unique(df['Head'])):
#                 try:
#                     pval = pearsonr(df.loc[df['Head']==c, edge_name_i], df.loc[df['Head']==c, tar])[1]
#                     axs[i+3*j].text(k, axs[i+3*j].get_ylim()[0] - (.15 if not j else .3), f'p={pval:.1e}', ha='center', va='center')
#                 except: continue

#     fig.savefig(f'../plots/group_differential_expression_{contrast_name}_{column}_{comparison}_{target}_{target_comparison}.pdf', format='pdf', transparent=True, backend='cairo')


### 4CD Linkage Cluster Enrichment

In [22]:
# for contrast_name, _, column, _, target, target_comparison in contrast_groupings:
#     for tar in (target, target_comparison):
#         # Get subject ids
#         group = None  # contrast_group
#         if group is None:
#             # Population
#             contrast_subjects = sum([v for k, v in get_contrast(contrast_name).items()], [])
#         else:
#             # Group
#             contrast_subjects = get_contrast(contrast_name)[group]

#         # Modify stack to include only contrast
#         df = np.nan_to_num(attention_stack[:, np.argwhere(np.array(columns)==column)[0][0], [s in contrast_subjects for s in subject_ids]])
#         new_subject_ids = [s for s in subject_ids if s in contrast_subjects]
#         df = pd.DataFrame(df, index=all_edges, columns=new_subject_ids)

#         # Get 100 most variant edges
#         df = df.iloc[df.to_numpy().var(axis=1).argsort()[::-1][:100]]

#         # Cluster
#         labels = KMeans(n_clusters=10, n_init=10).fit_predict(df.to_numpy().T)
#         labels += 1

#         # Get phenotypes
#         pheno = [meta.iloc[np.argwhere(meta['SubID'] == sid)[0][0]][tar] for sid in new_subject_ids]

#         # Format results
#         df = pd.DataFrame({'Cluster': labels, tar: pheno}, index=new_subject_ids)
#         df['count'] = 1
#         df = df.pivot_table(index='Cluster', columns=tar, values='count', aggfunc='sum').fillna(0)

#         # Transform to hypergeometric
#         df_np = df.to_numpy()
#         df_np_new = np.zeros_like(df_np)
#         for i, j in product(*[range(k) for k in df.shape]):
#             # i - cluster, j - target
#             dist = hypergeom(df_np.sum(), df_np[:, j].sum(), df_np[i, :].sum())
#             # Calculate probability of overrepresentation
#             df_np_new[i, j] = 1 - dist.cdf(df_np[i, j])
#         with np.errstate(divide='ignore'):
#             df_np_new = -np.log10(df_np_new)
#             df_np_new[np.isinf(df_np_new)] = np.nan
#         df = pd.DataFrame(df_np_new, index=df.index, columns=df.columns)

#         # Plot
#         fig, axs = get_mosaic([list(range(1))], scale=9)
#         sns.heatmap(df, vmin=0, vmax=3, cmap='rocket_r', cbar_kws={'label': '-log10(p)'}, ax=axs[0])
#         # plt.tight_layout()
#         fig.savefig(f'../plots/group_linkage_cluster_{contrast_name}_{column}_{tar}.pdf', format='pdf', transparent=True, backend='cairo')


### 4E Aggregate Graph Enrichment (MANUAL)

In [33]:
# # NOTE: Only top 100 are taken for aggregate due to memory concerns
# for contrast_name, group, column, _, _, _ in contrast_groupings:
#       # Load contrast
#       np.random.seed(42)
#       contrast_subjects = get_contrast(contrast_name)
#       gs = {
#             gname: concatenate_graphs(*[
#                   compute_graph(g)
#                   for g in load_many_graphs(np.random.choice(sids, 100, replace=False))[0]
#             ])
#             for gname, sids in contrast_subjects.items()
#       }

#       # Split into groups
#       # TODO: Make more general, perhaps add comparison group to arguments
#       g1_name = group
#       g1 = gs[g1_name]
#       g2_name = 'Control'
#       g2 = gs[g2_name]

#       # Get unique TFs
#       df = compare_graphs_enrichment(
#             g1,
#             g2,
#             sid_1=g1_name,
#             sid_2=g2_name,
#             nodes=synthetic_nodes_of_interest,
#             threshold=.01)

#       # Save to file
#       df.to_csv(f'../plots/genes_{contrast_name}_{group}_{column}.csv', index=False)


No threshold provided, using threshold of 0.05016567523128692.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 299268/299268 [00:00<00:00, 340656.93it/s]


Filtered from 4693 vertices and 50603 edges to 1565 vertices and 5755 edges via common edge filtering.
No threshold provided, using threshold of 0.05386086725079708.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 253981/253981 [00:00<00:00, 339890.80it/s]


Filtered from 4704 vertices and 49517 edges to 1365 vertices and 4839 edges via common edge filtering.
No threshold provided, using threshold of 0.05016567523128692.
Removing duplicate edges...


...

Filtered from 4704 vertices and 49517 edges to 1365 vertices and 4839 edges via common edge filtering.
No threshold provided, using threshold of 0.05016567523128692.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 299268/299268 [00:00<00:00, 327908.08it/s]


Filtered from 4693 vertices and 50603 edges to 1565 vertices and 5755 edges via common edge filtering.
No threshold provided, using threshold of 0.05386086725079708.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 253981/253981 [00:00<00:00, 340098.61it/s]


Filtered from 4704 vertices and 49517 edges to 1365 vertices and 4839 edges via common edge filtering.


In [35]:
# # Enrichment
# for contrast_name, group, column, _, _, _ in contrast_groupings:
#     # MANUAL PROCESSING
#     # Run the output from above on Metascape as multiple gene list and perform
#     # enrichment.  From the all-in-one ZIP file, save the file from
#     # Enrichment_QC/GO_DisGeNET as '../plot/disgenet_{subject_id_1}_{subject_id_2}_{column}.csv' and
#     # Overlap_circos/CircosOverlapByGene.svg as '../plot/overlap_{subject_id_1}_{subject_id_2}_{column}.svg'

#     # Get enrichment
#     enrichment_file = f'../plots/disgenet_{contrast_name}_{group}_{column}.csv'
#     if enrichment_file is None: continue
#     enrichment = pd.read_csv(enrichment_file)

#     # Format
#     enrichment = format_enrichment(enrichment)

#     # Plot
#     fig, axs = get_mosaic([[0]*2], scale=9)
#     pl = sns.scatterplot(
#         enrichment,
#         x='Gene Set', y='Description',
#         size='-log10(p)',
#         color='black',
#         ax=axs[0])
#     # Formatting
#     pl.grid()
#     plt.xticks(rotation=90)
#     pl.set_aspect('equal', 'box')
#     pl.legend(title='-log10(p)', bbox_to_anchor=(1.2, 1.05))
#     # Zoom X1
#     margin = .5
#     min_xlim, max_xlim = pl.get_xlim()
#     min_xlim -= margin; max_xlim += margin
#     pl.set(xlim=(min_xlim, max_xlim))
#     # Save
#     fig.savefig(f'../plots/group_enrichment_{contrast_name}_{group}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
